In [7]:
# Process s1..s6, build HAR windows, filter EEG by windows, report totals

from pathlib import Path
from datetime import timedelta
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

try:
    from tqdm.auto import tqdm
    use_tqdm = True
except Exception:
    use_tqdm = False

# ===== settings =====
SUBJECTS = [1, 2, 3, 4, 5, 6]
HAR_BASE = "/home/jupyter-yin10/EEG_HAR/data/har_convert"
EEG_BASE = "/home/jupyter-yin10/EEG_HAR/NEW/data/1_EEG_25hz_clean"
OUT_BASE = "/home/jupyter-yin10/EEG_HAR/NEW/data/2_EEG_window_matched"
YEAR_MIN, YEAR_MAX = 2025, 2025
USE_DOMINANT_DAY = True

TZ_LOCAL = "America/Indiana/Indianapolis"
EEG_TIME_IS_UTC = True  # set to False if EEG timestamps are already local

# ===== helpers shared =====
def safe_read_csv(path):
    try:
        return pd.read_csv(path, dtype=str, encoding="utf-8-sig", na_filter=False)
    except UnicodeDecodeError:
        return pd.read_csv(path, dtype=str, encoding="utf-8", na_filter=False)

def normalize_leftmost_timestamp(series):
    s = series.astype(str)
    s = s.str.replace("\u00A0", " ", regex=False)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    no_sec_mask = s.str.match(r"^\d{4}/\d{1,2}/\d{1,2}\s+\d{1,2}:\d{2}$")
    s.loc[no_sec_mask] = s.loc[no_sec_mask] + ":00"
    return s

def parse_leftmost_timestamp(series):
    s = normalize_leftmost_timestamp(series)
    dt = pd.to_datetime(s, format="%Y/%m/%d %H:%M:%S", errors="coerce")
    return dt

def pick_core_times(dt):
    dt = dt.dropna()
    if dt.empty:
        return dt
    yr = (dt.dt.year >= YEAR_MIN) & (dt.dt.year <= YEAR_MAX)
    dt = dt[yr]
    if dt.empty:
        return dt
    if USE_DOMINANT_DAY:
        dom = dt.dt.date.value_counts().idxmax()
        dt = dt[dt.dt.date == dom]
    return dt

def normalize_ts_text(s):
    s = s.astype(str)
    s = s.str.replace("\u00A0", " ", regex=False)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    no_sec = s.str.match(r"^\d{4}/\d{1,2}/\d{1,2}\s+\d{1,2}:\d{2}$")
    s.loc[no_sec] = s.loc[no_sec] + ":00"
    return s

def try_parse_dt(series):
    s = normalize_ts_text(series)
    dt = pd.to_datetime(s, format="%Y/%m/%d %H:%M:%S", errors="coerce")
    mask = dt.isna()
    if mask.any():
        dt2 = pd.to_datetime(s[mask], errors="coerce")
        dt.loc[mask] = dt2
    return dt

def find_eeg_timestamp_column(df):
    best_col, best_rate = None, -1.0
    for col in df.columns:
        dt = try_parse_dt(df[col])
        rate = dt.notna().mean()
        if rate > best_rate:
            best_rate, best_col = rate, col
    return best_col if best_rate >= 0.60 else None

def per_window_mask_tz(times_tz: pd.Series, windows_df: pd.DataFrame):
    keep = pd.Series(False, index=times_tz.index)
    first_hit = pd.Series(np.nan, index=times_tz.index)
    counts = []
    for _, w in windows_df.iterrows():
        s = w["start"]
        e = w["end"]
        m = (times_tz >= s) & (times_tz <= e)
        counts.append(int(m.sum()))
        first_hit.loc[m & first_hit.isna()] = int(w["window_id"])
        keep |= m
    return keep, first_hit, counts

# ===== runners for a single subject =====
def build_windows_for_subject(subj: int) -> Path:
    har_dir = Path(HAR_BASE) / f"s{subj}"
    out_dir = Path(OUT_BASE) / f"s{subj}"
    out_dir.mkdir(parents=True, exist_ok=True)
    windows_csv = out_dir / f"har_time_windows_s{subj}.csv"

    rows = []
    for p in sorted(har_dir.glob("*.csv")):
        if p.name == windows_csv.name:
            continue
        df = safe_read_csv(p)
        if df.shape[1] == 0:
            continue
        dt = parse_leftmost_timestamp(df.iloc[:, 0])
        dt = pick_core_times(dt)
        if dt.empty:
            print(f"[s{subj}] [WARN] {p.name}, no valid timestamps in {YEAR_MIN}-{YEAR_MAX}, skipped.")
            continue

        start_raw = dt.min()
        end_raw = dt.max()
        start_expanded = start_raw - timedelta(seconds=10)
        end_expanded = end_raw + timedelta(seconds=10)

        rows.append({
            "window_id": len(rows) + 1,
            "har_file": p.name,
            "start": start_expanded.strftime("%Y-%m-%d %H:%M:%S"),
            "end": end_expanded.strftime("%Y-%m-%d %H:%M:%S")
        })

    win_df = pd.DataFrame(rows, columns=["window_id", "har_file", "start", "end"])
    if win_df.empty:
        raise RuntimeError(f"[s{subj}] No HAR windows built.")
    win_df.to_csv(windows_csv, index=False, encoding="utf-8")
    print(f"[s{subj}] Saved {len(win_df)} HAR time windows -> {windows_csv}")
    return windows_csv

def filter_eeg_for_subject(subj: int, windows_csv: Path) -> pd.DataFrame:
    eeg_dir = Path(EEG_BASE) / f"s{subj}"
    out_dir = Path(OUT_BASE) / f"s{subj}"
    out_dir.mkdir(parents=True, exist_ok=True)

    win_df = pd.read_csv(windows_csv, dtype={"window_id": int, "har_file": str, "start": str, "end": str})
    if win_df.empty:
        raise RuntimeError(f"[s{subj}] Windows CSV is empty.")

    win_df["start"] = pd.to_datetime(win_df["start"], errors="coerce").dt.tz_localize(TZ_LOCAL)
    win_df["end"]   = pd.to_datetime(win_df["end"],   errors="coerce").dt.tz_localize(TZ_LOCAL)

    eeg_csvs = sorted(eeg_dir.glob("*.csv"))
    if not eeg_csvs:
        print(f"[s{subj}] No EEG CSVs found in {eeg_dir}")
        return pd.DataFrame(columns=["subject","eeg_file","status","rows_before","rows_after","pct_in_window"])

    iterator = tqdm(eeg_csvs, desc=f"s{subj} filtering", unit="file") if use_tqdm else eeg_csvs
    summary_rows = []

    for eeg_path in iterator:
        df = safe_read_csv(eeg_path)
        before = len(df)
        if df.empty:
            out_name = f"{eeg_path.stem}_filtered.csv"
            df.to_csv(out_dir / out_name, index=False, encoding="utf-8")
            summary_rows.append([subj, eeg_path.name, "EMPTY FILE", before, before, 100.0])
            continue

        ts_col = find_eeg_timestamp_column(df)
        if ts_col is None:
            out_name = f"{eeg_path.stem}_filtered.csv"
            df.to_csv(out_dir / out_name, index=False, encoding="utf-8")
            summary_rows.append([subj, eeg_path.name, "NO TS COL, wrote full file", before, before, 100.0])
            continue

        times = try_parse_dt(df[ts_col])
        if EEG_TIME_IS_UTC:
            times = times.dt.tz_localize("UTC", nonexistent="NaT", ambiguous="NaT").dt.tz_convert(TZ_LOCAL)
        else:
            times = times.dt.tz_localize(TZ_LOCAL, nonexistent="NaT", ambiguous="NaT")

        keep_mask, first_id, counts = per_window_mask_tz(times, win_df)

        if not keep_mask.any():
            filtered = df.copy()
            status = "NO MATCHES, wrote full file"
            after = before
            pct = 100.0
        else:
            filtered = df[keep_mask].copy()
            filtered.insert(0, "matched_window_id", first_id[keep_mask].astype("Int64"))
            status = "OK"
            after = len(filtered)
            pct = round(after / before * 100.0, 2)

        out_name = f"{eeg_path.stem}_filtered.csv"
        filtered.to_csv(out_dir / out_name, index=False, encoding="utf-8")
        summary_rows.append([subj, eeg_path.name, status, before, after, pct])

    sum_df = pd.DataFrame(summary_rows, columns=["subject","eeg_file","status","rows_before","rows_after","pct_in_window"])
    return sum_df

# ===== run all subjects, then print totals =====
all_summaries = []
for s in SUBJECTS:
    windows_csv = build_windows_for_subject(s)
    sum_df = filter_eeg_for_subject(s, windows_csv)
    if not sum_df.empty:
        all_summaries.append(sum_df)

if all_summaries:
    all_df = pd.concat(all_summaries, ignore_index=True)
else:
    all_df = pd.DataFrame(columns=["subject","eeg_file","status","rows_before","rows_after","pct_in_window"])

# Per subject totals
if not all_df.empty:
    by_subj = all_df.groupby("subject", as_index=False).agg(
        files=("eeg_file", "count"),
        rows_before=("rows_before", "sum"),
        rows_after=("rows_after", "sum")
    )
    by_subj["pct_kept"] = (by_subj["rows_after"] / by_subj["rows_before"] * 100).round(2)
    by_subj["pct_dropped"] = (100 - by_subj["pct_kept"]).round(2)

    # Overall totals
    total_before = int(by_subj["rows_before"].sum())
    total_after = int(by_subj["rows_after"].sum())
    pct_kept = round(total_after / total_before * 100, 2) if total_before > 0 else 0.0
    pct_dropped = round(100 - pct_kept, 2)

    print("\n=== Per subject totals ===")
    print(by_subj[["subject","files","rows_before","rows_after","pct_kept","pct_dropped"]].to_string(index=False))

    print("\n=== Overall totals across s1 to s6 ===")
    print(f"Rows before, {total_before:,}")
    print(f"Rows after,  {total_after:,}")
    print(f"Percent kept, {pct_kept:.2f}%")
    print(f"Percent dropped, {pct_dropped:.2f}%")

    # Optional, save summaries
    out_summary_dir = Path(OUT_BASE)
    by_subj.to_csv(out_summary_dir / "summary_by_subject.csv", index=False)
    all_df.to_csv(out_summary_dir / "summary_all_files.csv", index=False)
else:
    print("No summaries to report.")


[s1] [WARN] tmp_windows.csv, no valid timestamps in 2025-2025, skipped.
[s1] Saved 10 HAR time windows -> /home/jupyter-yin10/EEG_HAR/NEW/data/2_EEG_window_matched/s1/har_time_windows_s1.csv


s1 filtering:   0%|          | 0/10 [00:00<?, ?file/s]

[s2] Saved 10 HAR time windows -> /home/jupyter-yin10/EEG_HAR/NEW/data/2_EEG_window_matched/s2/har_time_windows_s2.csv


s2 filtering:   0%|          | 0/10 [00:00<?, ?file/s]

[s3] Saved 10 HAR time windows -> /home/jupyter-yin10/EEG_HAR/NEW/data/2_EEG_window_matched/s3/har_time_windows_s3.csv


...

[s5] Saved 4 HAR time windows -> /home/jupyter-yin10/EEG_HAR/NEW/data/2_EEG_window_matched/s5/har_time_windows_s5.csv


s5 filtering:   0%|          | 0/4 [00:00<?, ?file/s]

[s6] Saved 10 HAR time windows -> /home/jupyter-yin10/EEG_HAR/NEW/data/2_EEG_window_matched/s6/har_time_windows_s6.csv


s6 filtering:   0%|          | 0/10 [00:00<?, ?file/s]


=== Per subject totals ===
 subject  files  rows_before  rows_after  pct_kept  pct_dropped
       1     10       147593      126616     85.79        14.21
       2     10       111174       95169     85.60        14.40
       3     10       112539       91923     81.68        18.32
       4     10       108759       86554     79.58        20.42
       5      4        29268       28292     96.67         3.33
       6     10       106565       89076     83.59        16.41

=== Overall totals across s1 to s6 ===
Rows before, 615,898
Rows after,  517,630
Percent kept, 84.04%
Percent dropped, 15.96%
